In [8]:
import requests
import pandas as pd

BASE_URL = "http://127.0.0.1:5000"

def get_auth_token(username, password):
    """Fetch JWT token for authentication."""
    auth_data = {"username": username, "password": password}
    try:
        token_response = requests.post(f"{BASE_URL}/token", data=auth_data)
        if token_response.status_code == 200:
            return token_response.json().get("access_token")
        print(f"Failed to get token. Response: {token_response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error getting token: {str(e)}")
    return None

def fetch_data(endpoint, token):
    """Fetch data from a given API endpoint."""
    headers = {"Authorization": f"Bearer {token}"}
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=headers)
        if response.status_code == 200:
            return pd.DataFrame(response.json())
        print(f"Failed to fetch data from {endpoint}. Response: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {endpoint}: {str(e)}")
    return None

def main():
    username = "justin"
    password = "password123"

    token = get_auth_token(username, password)
    if not token:
        print("Authentication failed. Exiting...")
        return

    print("Fetching datasets...")
    patients_df = fetch_data("patients", token)
    billing_df = fetch_data("billing", token)
    medical_records_df = fetch_data("records", token)

    if patients_df is None:
        print("Failed to fetch 'patients' data.")
    else:
        print(f"\n'patients' Dataset Shape: {patients_df.shape}")
        print(f"\n'patients' Data Description:\n{patients_df.describe()}")
        print(f"\n'patients' Data Types:\n{patients_df.dtypes}")

        
        patients_df.to_csv('patients_data.csv', index=False)
        print("Exported 'patients' data to 'patients_data.csv'.")

    if billing_df is None:
        print("Failed to fetch 'billing' data.")
    else:
        print(f"\n'billing' Dataset Shape: {billing_df.shape}")
        print(f"\n'billing' Data Description:\n{billing_df.describe()}")
        print(f"\n'billing' Data Types:\n{billing_df.dtypes}")
        # Export billing data to CSV
        billing_df.to_csv('billing_data.csv', index=False)
        print("Exported 'billing' data to 'billing_data.csv'.")

    
    if patients_df is not None and billing_df is not None:
        merged_df = pd.merge(patients_df, billing_df, left_on='id', right_on='patient_id', how='inner')
        print(f"\n'Merged Dataset Shape: {merged_df.shape}")
        print(f"\n'Merged Dataset Head:\n{merged_df.head()}")
        print(f"\n 'Merged Dataset Tail: \n{merged_df.tail()}'")

        
        merged_df.to_csv('patients_billing.csv', index=False)
        print("Exported merged data to 'patients_billing.csv'.")

if __name__ == "__main__":
    main()


Fetching datasets...

'patients' Dataset Shape: (500000, 5)

'patients' Data Description:
                  id
count  500000.000000
mean   250000.500000
std    144337.711634
min         1.000000
25%    125000.750000
50%    250000.500000
75%    375000.250000
max    500000.000000

'patients' Data Types:
name             object
dob              object
contact_info     object
id                int64
date_attended    object
dtype: object
Exported 'patients' data to 'patients_data.csv'.

'billing' Dataset Shape: (500000, 4)

'billing' Data Description:
          patient_id     total_cost             id
count  500000.000000  500000.000000  500000.000000
mean   199894.617296    6273.396360  250000.500000
std    115448.543339    3595.796123  144337.711634
min         0.000000      50.050000       1.000000
25%    100225.500000    3155.310000  125000.750000
50%    199889.000000    6268.905000  250000.500000
75%    299831.250000    9387.877500  375000.250000
max    400000.000000   12499.990000  50